<a href="https://colab.research.google.com/github/pikanaeri/plm-model-comparison/blob/main/efam-performance/EFAM_Embedding_EsmC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Starting Procedure (Computer Setup)

In [ ]:
sudo apt-get update -qq
sudo apt-get install -y tmux
tmux

In [ ]:
sudo apt-get update
sudo apt-get install bzip2 libxml2-dev
sudo apt upgrade
sudo apt-get install wget
wget https://repo.anaconda.com/archive/Anaconda3-2022.05-Linux-x86_64.sh
bash Anaconda3-2022.05-Linux-x86_64.sh
rm Anaconda3-2022.05-Linux-x86_64.sh

In [ ]:
sudo fallocate -l 15G /swapfile
sudo chmod 600 /swapfile
ls -lh /swapfile
sudo mkswap /swapfile
sudo swapon /swapfile

#@markdown Append "/swapfile swap swap defaults 0 0" to the /etc/fstab file using a text editor of your choice
sudo nano /etc/fstab

/swapfile swap swap defaults 0 0

In [ ]:
sudo apt-get install git
sudo apt install python3-pip

python3 --version
pip3 --version

gsutil cp -r gs://plm-model-comparison/ .

mkdir final_embeddings

SyntaxError: invalid syntax (<ipython-input-1-3b0e3c477eff>, line 1)

In [ ]:
!pip install transformers
!pip3 install torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


##Embedding using EFAM

In [ ]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig
import torch
import re
import time
import gc
import numpy as np
import pickle as pkl

# In[1]:


import pandas as pd


# In[12]:


x = pd.read_csv('efam_phrog_e10.csv', index_col=0)
y = pd.read_csv('PHROG_index_revised_v4_10292022.csv', index_col=0)


# In[18]:


d = dict(zip(list(y.index), y['revised_category_v4']))


# In[22]:


x['label'] = [d[p] for p in x['phrog#']]


# In[27]:


xx = x[x['label'] != 'unknown function']


# In[33]:


len(set(xx['cluster']))

print(len(set(xx['cluster'])))
print(xx)

In [ ]:
#@title Creating model and setting device to CPU if available
client = ESMC.from_pretrained("esmc_600m").to("cuda") # or "cpu"

In [ ]:
#@title Embedding Families
#1. Get sequences here
def get_seqs(File_Name):
  names = []
  seqs = [] #should match up with the efam_phrog_e10 file
  reader = open(File_Name, "r")
  name = reader.readline().strip()
  while True: #The sequences are broken up by new lines!!!
    if name == '':
      break
    sequence = ""
    read = reader.readline().strip()
    while read != '' and read[0] != '>':
      sequence += read
      read = reader.readline().strip()
    names.append(name[1:])
    seqs.append(sequence)
    name = read
  reader.close()
  return names, seqs


names, seqs = get_seqs("dereplicated_filtered_proteins_efam_downloaded_10162022.faa")


#2. Embed families here

def embed_family(cluster_name, max_sz=-1):
  embeddings = []
  nms = []
  seq_nms = x[x['cluster']==cluster_name].index.tolist()
  for i in seq_nms:
    seqs[i] = seqs[i].upper()
    print(seqs[i])
    if max_sz == -1 or len(seqs[i]) <= max_sz:
      nms.append(names[i])
      protein = ESMProtein(sequence=seqs[i])
      protein_tensor = client.encode(protein)
      logits_output = client.logits(
        protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
      )
      embeddings.append(np.mean(logits_output.embeddings[0].cpu().numpy(), axis=0))
      gc.collect()
      torch.cuda.empty_cache()
  print(seq_nms)
  print(embeddings)
  pkl.dump(nms, open(f"final_embeddings_uppercase/{cluster_name}.txt", "wb"))
  pkl.dump(embeddings, open(f"final_embeddings_uppercase/{cluster_name}.pkl", "wb"))
  return nms, embeddings


#Navigate PHROGs directory here
def nav_dir(sz_cap):
  dct = {}
  clusters = sorted(set(xx['cluster']))
  print(clusters)
  for c in clusters:
    print("embedding ", c)
    s, e = embed_family(c, max_sz=sz_cap)
    for i in range(len(s)):
      dct[s[i]] = e[i]
  f = open("identifier_to_vector_protbert_bdf_esmc_12112024.pkl", "wb")
  pkl.dump(dct, f)
  f.close()

print("beginning procedure")
nav_dir(sz_cap=5096)

SyntaxError: unterminated string literal (detected at line 49) (<ipython-input-1-c88580dd61bd>, line 48)